<a href="https://colab.research.google.com/github/Maiven/AI_DATA/blob/main/data%20engineering/data%20pipeline/1_Naver_shopping_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [144]:
import os
import sys
import json

import urllib.request
client_id = "gdzCRwvapR7vaYbRlzBM"
client_secret = "AM6Ui24Luh"
url = "https://openapi.naver.com/v1/datalab/shopping/categories";
body = "{\"startDate\":\"2020-01-01\"," \
            "\"endDate\":\"2020-01-10\"," \
            "\"timeUnit\":\"date\"," \
            "\"category\":[{\"name\":\"패션의류\",\"param\":[\"50000000\"]}]}"
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)
    

def get_shop(client_id, client_secret):
    # 아이디,시크릿 기입
    endpoint = 'https://openapi.naver.com/v1/datalab/shopping/categories'
    bodys = "{\"startDate\":\"2020-01-01\"," \
            "\"endDate\":\"2020-01-10\"," \
            "\"timeUnit\":\"date\"," \
            "\"category\":[{\"name\":\"패션의류\",\"param\":[\"50000000\"]}]}"

    # url 정보 기입
    requests = urllib.request.Request(endpoint)

    # header 정보 기입
    requests.add_header("X-Naver-Client-Id", client_id)
    requests.add_header("X-Naver-Client-Secret", client_secret)
    requests.add_header("Content-Type", "application/json")

    response = urllib.request.urlopen(requests, data=bodys.encode('utf-8'))
    
    # 에러 발생할 경우 예외 처리
    try:
        response = urllib.request.urlopen(requests, data=bodys.encode('utf-8'))
    except:
        logging.error(response.read())
        sys.exit(1)

    response = urllib.request.urlopen(requests, data=bodys.encode('utf-8'))
	
    # 제대로 연결이 되었을 때 api 의 내용을 출력하기
    if response.getcode() == 200:
        response_body = response.read()
        contents = response_body.decode('utf-8')
    else:
        print(response.getcode())
        print(response.read())
        sys.exit(1)
        
    return contents
 

def main():
    
    # aws 의 mysql 과 직접 연결
    try:
        # mysql 과 직접 연결
        conn = pymysql.connect(host = 'database-4.castm60mjxbf.ap-northeast-2.rds.amazonaws.com', user='code_cho2', passwd='eogml5132', db='codechodb', use_unicode=True, charset='utf8')
        cursor = conn.cursor()
    except Exception as e:
        print(e)
        logging.error("could not connect to RDS")
        sys.exit(1)

    shops = get_shop(client_id,client_secret)

    # json 으로 변환하기
    raw = json.loads(shops)

    # date, ratio 를 추출
    datas = raw['results'][0]['data']

    # date 와 ratio 를 aws에 넣기 전, 리스트에 담기
    shop_content = []
    for data in datas:
        shop_content.append(
            {
                'dates' : data['period'],
                'ratios' : data['ratio']
            }
        )
    print(shop_content)

    for content in shop_content: 
      sql = "INSERT INTO shop_20 VALUES (%(dates)s, %(ratios)s);"


    
    cursor.executemany(sql, shop_content)
    conn.commit() 
    print(cursor.rowcount, "record inserted.")

    sys.exit(0)    
main()

{"startDate":"2020-01-01","endDate":"2020-01-10","timeUnit":"date","results":[{"title":"패션의류","category":["50000000"],"data":[{"period":"2020-01-01","ratio":89.00652},{"period":"2020-01-02","ratio":87.14725},{"period":"2020-01-03","ratio":80.25985},{"period":"2020-01-04","ratio":88.98288},{"period":"2020-01-05","ratio":100},{"period":"2020-01-06","ratio":90.04795},{"period":"2020-01-07","ratio":89.18886},{"period":"2020-01-08","ratio":85.80307},{"period":"2020-01-09","ratio":82.07851},{"period":"2020-01-10","ratio":73.47124}]}]}
[{'dates': '2020-01-01', 'ratios': 89.00652}, {'dates': '2020-01-02', 'ratios': 87.14725}, {'dates': '2020-01-03', 'ratios': 80.25985}, {'dates': '2020-01-04', 'ratios': 88.98288}, {'dates': '2020-01-05', 'ratios': 100}, {'dates': '2020-01-06', 'ratios': 90.04795}, {'dates': '2020-01-07', 'ratios': 89.18886}, {'dates': '2020-01-08', 'ratios': 85.80307}, {'dates': '2020-01-09', 'ratios': 82.07851}, {'dates': '2020-01-10', 'ratios': 73.47124}]
10 record inserted.

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [42]:
pip install pymysql

     |████████████████████████████████| 51kB 2.9MB/s 


In [145]:
import pandas as pd
import numpy as np
import pymysql
import logging

# AWS mysql 연결하기 

host = "database-4.castm60mjxbf.ap-northeast-2.rds.amazonaws.com"
username = "code_cho2"
password = "eogml5132"
database = "codechodb"

def get_data(host,username,database,password):
    try:
      conn  = pymysql.connect(host = host, user=username, db=database, passwd=password, use_unicode=True, charset='utf8', cursorclass=pymysql.cursors.DictCursor)
      cursor = conn.cursor()
    except Exception as e:
      print(e)
      logging.error("could not connect to AWS RDS mysql. Please to let engineers know that!")
      sys.exit(1)
    
    # rds -  데이터를 가져온다
    cursor.execute("select * from shop_20;")
    result = cursor.fetchall()
    conn.close()

    df = pd.DataFrame(result)
    return df

hah = get_data(host,username,database,password)
hah


,dates,ratios
0,2020-01-01,89
1,2020-01-02,87
2,2020-01-03,80
3,2020-01-04,89
4,2020-01-05,100
5,2020-01-06,90
6,2020-01-07,89
7,2020-01-08,86
8,2020-01-09,82
9,2020-01-10,73


NameError: ignored